In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time
import re

BASE_URL = "https://www.caf.com"

driver = webdriver.Chrome()
driver.get("https://www.caf.com/es/quienes-somos/proyectos/?page=3")
time.sleep(7)

# Scroll for lazy loading
for _ in range(4):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

projects = driver.find_elements(By.CLASS_NAME, "card__project")
details = driver.find_elements(By.CLASS_NAME, "card__project__details")

data = []

for i, card in enumerate(projects):
    proj_lines = card.text.strip().split("\n")
    # Find the first <a> inside the card for the project URL
    try:
        link_el = card.find_element(By.TAG_NAME, "a")
        url = link_el.get_attribute("href")
        # If the link is relative, add base
        if url and url.startswith("/"):
            url = BASE_URL + url
    except Exception:
        url = ""

    project = {
        "Country": proj_lines[0] if len(proj_lines) > 0 else "",
        "Project Name": proj_lines[1] if len(proj_lines) > 1 else "",
        "Number": "",
        "Approval Date": "",
        "Risk Type": "",
        "Financing Instrument": "",
        "Sector": "",
        "Operation Status": "",
        "Project URL": url,
    }
    # Parse details
    detail_text = details[i].text if i < len(details) else ""
    m_num = re.search(r"Número de operación:\s*([^\n]+)", detail_text)
    m_date = re.search(r"Fecha de aprobación:\s*([^\n]+)", detail_text)
    m_risk = re.search(r"Tipo de riesgo:\s*([^\|\n]+)", detail_text)
    m_fin = re.search(r"Instrumento de financiación:\s*([^\|\n]+)", detail_text)
    m_sector = re.search(r"Sector:\s*([^\n]+)", detail_text)

    if m_num:
        project["Number"] = m_num.group(1).strip()
    if m_date:
        project["Approval Date"] = m_date.group(1).strip()
    if m_risk:
        project["Risk Type"] = m_risk.group(1).strip()
    if m_fin:
        project["Financing Instrument"] = m_fin.group(1).strip()
    if m_sector:
        project["Sector"] = m_sector.group(1).strip()

    # Find status: next sibling div after details
    try:
        details_el = details[i]
        status_el = details_el.find_element(By.XPATH, 'following-sibling::*[1]')
        status_text = status_el.text.strip().upper()
        if "OPERACIÓN" in status_text:
            # Translate to English if desired
            if "ACTIVA" in status_text:
                project["Operation Status"] = "Active operation"
            elif "CERRADA" in status_text:
                project["Operation Status"] = "Operation closed"
            else:
                project["Operation Status"] = status_text.title()
        else:
            status_el2 = details_el.find_element(By.XPATH, 'following-sibling::*[2]')
            status_text2 = status_el2.text.strip().upper()
            if "OPERACIÓN" in status_text2:
                if "ACTIVA" in status_text2:
                    project["Operation Status"] = "Active operation"
                elif "CERRADA" in status_text2:
                    project["Operation Status"] = "Operation closed"
                else:
                    project["Operation Status"] = status_text2.title()
    except Exception:
        pass

    data.append(project)

driver.quit()

df = pd.DataFrame(data)
df.to_excel("caf_projects_with_url.xlsx", index=False)
print("Done! Saved as caf_projects_with_url.xlsx")


Done! Saved as caf_projects_with_url.xlsx


In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

# 1. Load your previous Excel file with project URLs
df_main = pd.read_excel("caf_projects_with_url.xlsx")

# 2. Set up Selenium (headless mode is fine here)
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

all_rows = []

for idx, row in df_main.iterrows():
    url = row['Project URL']
    driver.get(url)
    time.sleep(5)  # Adjust as needed for page load

    # Find the financing table (it may be missing for some projects)
    fin_data = {}
    try:
        table_rows = driver.find_elements(By.CSS_SELECTOR, ".project__fin-table__2-col")
        for trow in table_rows:
            cells = trow.find_elements(By.TAG_NAME, "p")
            if len(cells) == 2:
                key = cells[0].text.strip()
                val = cells[1].text.strip()
                fin_data[key] = val
    except Exception as e:
        print(f"Error at {url}: {e}")

    # Merge original columns + new indicators for this project
    out = dict(row)
    # Add standard finance fields (edit as needed)
    for col in [
        "Costo del proyecto", "Préstamo CAF", "Desembolsado CAF",
        "Aporte local", "Otros aportes", "Detalle otros aportes", "Moneda de la inversión"
    ]:
        out[col] = fin_data.get(col, "")

    all_rows.append(out)

driver.quit()

df_final = pd.DataFrame(all_rows)
df_final.to_excel("caf_projects_with_financials.xlsx", index=False)
print("All done! Saved as caf_projects_with_financials.xlsx")


All done! Saved as caf_projects_with_financials.xlsx


In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time
import re
from deep_translator import GoogleTranslator

BASE_URL = "https://www.caf.com"
NUM_PAGES = 46  # There are 46 pages

def translate_text(text, src='es', dest='en'):
    # Avoid translating empty strings or None
    if not text or pd.isnull(text):
        return ""
    try:
        return GoogleTranslator(source=src, target=dest).translate(text)
    except Exception:
        return text  # Return original if translation fails

all_data = []

driver = webdriver.Chrome()
for page in range(1, NUM_PAGES + 1):
    print(f"Processing page {page}...")
    url = f"https://www.caf.com/es/quienes-somos/proyectos/?page={page}"
    driver.get(url)
    time.sleep(7)
    # Scroll for lazy loading
    for _ in range(4):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

    projects = driver.find_elements(By.CLASS_NAME, "card__project")
    details = driver.find_elements(By.CLASS_NAME, "card__project__details")

    for i, card in enumerate(projects):
        proj_lines = card.text.strip().split("\n")
        # Find the first <a> inside the card for the project URL
        try:
            link_el = card.find_element(By.TAG_NAME, "a")
            project_url = link_el.get_attribute("href")
            if project_url and project_url.startswith("/"):
                project_url = BASE_URL + project_url
        except Exception:
            project_url = ""

        country = proj_lines[0] if len(proj_lines) > 0 else ""
        project_name = proj_lines[1] if len(proj_lines) > 1 else ""

        # Parse details
        detail_text = details[i].text if i < len(details) else ""
        m_num = re.search(r"Número de operación:\s*([^\n]+)", detail_text)
        m_date = re.search(r"Fecha de aprobación:\s*([^\n]+)", detail_text)
        m_risk = re.search(r"Tipo de riesgo:\s*([^\|\n]+)", detail_text)
        m_fin = re.search(r"Instrumento de financiación:\s*([^\|\n]+)", detail_text)
        m_sector = re.search(r"Sector:\s*([^\n]+)", detail_text)

        num = m_num.group(1).strip() if m_num else ""
        approval_date = m_date.group(1).strip() if m_date else ""
        risk_type = m_risk.group(1).strip() if m_risk else ""
        financing_instr = m_fin.group(1).strip() if m_fin else ""
        sector = m_sector.group(1).strip() if m_sector else ""

        # Find status: next sibling div after details
        operation_status = ""
        try:
            details_el = details[i]
            status_el = details_el.find_element(By.XPATH, 'following-sibling::*[1]')
            status_text = status_el.text.strip().upper()
            if "OPERACIÓN" in status_text:
                if "ACTIVA" in status_text:
                    operation_status = "Active operation"
                elif "CERRADA" in status_text:
                    operation_status = "Operation closed"
                else:
                    # Translate any other statuses
                    operation_status = translate_text(status_text, src='es', dest='en').title()
            else:
                status_el2 = details_el.find_element(By.XPATH, 'following-sibling::*[2]')
                status_text2 = status_el2.text.strip().upper()
                if "OPERACIÓN" in status_text2:
                    if "ACTIVA" in status_text2:
                        operation_status = "Active operation"
                    elif "CERRADA" in status_text2:
                        operation_status = "Operation closed"
                    else:
                        operation_status = translate_text(status_text2, src='es', dest='en').title()
        except Exception:
            pass

        # Translate all string fields except URL and number/date fields
        project = {
            "Country": translate_text(country),
            "Project Name": translate_text(project_name),
            "Number": num,
            "Approval Date": approval_date,
            "Risk Type": translate_text(risk_type),
            "Financing Instrument": translate_text(financing_instr),
            "Sector": translate_text(sector),
            "Operation Status": operation_status,
            "Project URL": project_url,
        }

        all_data.append(project)

driver.quit()

df = pd.DataFrame(all_data)
df.to_excel("caf_projects_english.xlsx", index=False)
print("Done! Saved as caf_projects_english.xlsx")


Processing page 1...
Processing page 2...
Processing page 3...
Processing page 4...
Processing page 5...
Processing page 6...
Processing page 7...
Processing page 8...
Processing page 9...
Processing page 10...
Processing page 11...
Processing page 12...
Processing page 13...
Processing page 14...
Processing page 15...
Processing page 16...
Processing page 17...
Processing page 18...
Processing page 19...
Processing page 20...
Processing page 21...
Processing page 22...
Processing page 23...
Processing page 24...
Processing page 25...
Processing page 26...
Processing page 27...
Processing page 28...
Processing page 29...
Processing page 30...
Processing page 31...
Processing page 32...
Processing page 33...
Processing page 34...
Processing page 35...
Processing page 36...
Processing page 37...
Processing page 38...
Processing page 39...
Processing page 40...
Processing page 41...
Processing page 42...
Processing page 43...
Processing page 44...
Processing page 45...
Processing page 46.

In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time
from deep_translator import GoogleTranslator

# Mapping from Spanish to English for finance fields
FIN_KEY_TRANSLATE = {
    "Costo del proyecto": "Project Cost",
    "Préstamo CAF": "CAF Loan",
    "Desembolsado CAF": "CAF Disbursed",
    "Aporte local": "Local Contribution",
    "Otros aportes": "Other Contributions",
    "Detalle otros aportes": "Other Contribution Details",
    "Moneda de la inversión": "Investment Currency"
}

def translate_value(val, src='es', dest='en'):
    # Only translate if it's not a number or USD
    try:
        if val.strip().startswith("USD") or val.strip().replace('.', '').replace(',', '').isdigit():
            return val
        # Do not translate empty or already English values
        if not val or val.lower() in ["usd", "community"]:
            return val
        return GoogleTranslator(source=src, target=dest).translate(val)
    except Exception:
        return val

df_main = pd.read_excel("caf_projects_english.xlsx")
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

all_rows = []

for idx, row in df_main.iterrows():
    url = row['Project URL']
    driver.get(url)
    time.sleep(5)  # Wait for page load

    fin_data = {}
    try:
        table_rows = driver.find_elements(By.CSS_SELECTOR, ".project__fin-table__2-col")
        for trow in table_rows:
            cells = trow.find_elements(By.TAG_NAME, "p")
            if len(cells) == 2:
                key = cells[0].text.strip()
                val = cells[1].text.strip()
                key_en = FIN_KEY_TRANSLATE.get(key, key)  # Translate the header
                val_en = translate_value(val)
                fin_data[key_en] = val_en
    except Exception as e:
        print(f"Error at {url}: {e}")

    # Merge original columns + new indicators for this project
    out = dict(row)
    # Add English finance fields (keys from mapping)
    for en_col in FIN_KEY_TRANSLATE.values():
        out[en_col] = fin_data.get(en_col, "")

    all_rows.append(out)

driver.quit()

df_final = pd.DataFrame(all_rows)
df_final.to_excel("caf_projects_with_financials_english.xlsx", index=False)
print("All done! Saved as caf_projects_with_financials_english.xlsx")


All done! Saved as caf_projects_with_financials_english.xlsx
